In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
import random
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.tree import DecisionTreeClassifier

In [2]:
X, y = make_classification(n_samples=200, n_features=20, n_informative=2, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
eps = 1e-15

In [82]:
def true_positive(y_true, y_pred):
    return np.sum((y_true == 1) & (y_pred == 1)) 

def false_negative(y_true, y_pred):
    return np.sum((y_true == 1) & (y_pred == 0))

def false_positive(y_true, y_pred):
    return np.sum((y_true == 0) & (y_pred == 1))

def true_negative(y_true, y_pred):
    return np.sum((y_true == 0) & (y_pred == 0))

def accuracy(y_true, y_pred):
    tp = true_positive(y_true, y_pred)
    tn = true_negative(y_true, y_pred)
    return (tp + tn)/len(y_true)

def precision(y_true, y_pred):
    tp = true_positive(y_true, y_pred)
    fp = false_positive(y_true, y_pred)
    return tp/(tp+fp)

def recall(y_true, y_pred):
    tp = true_positive(y_true, y_pred)
    fn = false_negative(y_true, y_pred)
    return tp/(tp+fn)

def f1(y_true, y_pred):
    precision_value = precision(y_true, y_pred)
    recall_value = recall(y_true, y_pred)
    return (2*precision_value*recall_value)/(precision_value + recall_value)

In [4]:
def entropy(y): #энтропия
    entropy = 0
    classes = y.unique()
    for i in classes:
        count = (y==i).sum()
        p = count/len(y)
        entropy += p*np.log2(eps+p)
    return -entropy              

In [5]:
def IG(y, X, Q): #прирост информации 
    n = len(y)
    y1 = y.loc[X[X>Q].index]
    y2 = y.loc[X[X<=Q].index]

    S1 = entropy(y1)
    S2 = entropy(y2)
    
    S0 = entropy(y)
    return S0 - (S1*len(y1)/n + S2*len(y2)/n)   

In [6]:
def gini(y): #джинни
    gini = 0
    classes = y.unique()
    for i in classes:
        count = (y==i).sum()
        p = count/len(y)
        gini += p**2    
    return 1 - gini    

In [7]:
def gini_gain(y, X, Q): #джинни гейн

    y1 = y.loc[X[X > Q].index]
    y2 = y.loc[X[X <= Q].index]
    
    G1 = G(y1)
    G2 = G(y2)
    
 
    n1 = len(y1)
    n2 = len(y2)
    
    return G(y) - (G1*n1 + G2*n2)/n

In [8]:
df = pd.read_csv('data_banknote_authentication.txt', header=None)
df.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'target']
X, y = df.iloc[:,:4], df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
class TreeNode:
    def __init__(self, split_column, split_value, predicted_classes):
        self.split_column = split_column
        self.split_value = split_value
        self.predicted_classes = predicted_classes
        self.left = None
        self.right = None

In [10]:
class MyTreeClf:
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20, bins=None, criterion='entropy'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs if max_leafs > 1 else 2
        self.leafs_cnt = 0
        self.bins = bins
        if criterion == 'gini':
            self.criterion = globals()['gini_gain']
        else:
            self.criterion = globals()['IG']
        self.metric = globals()[criterion]    
        self.fi = {}
    
    def __str__(self):
        return f"MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}"
    
    
    def get_best_split(self, X, y):
        best_ig = 0
        best_Q = 0
        best_column_name = None
        prev_value = None
       
        if self.bins is None:
            for column in X.columns:
                column = X[column].sort_values()
                for index, value in column.items():
                    if prev_value != None:
                        Q = (prev_value + value)/2
                        ig = self.criterion(y, column, Q)
                        if ig > best_ig:
                            best_ig = ig
                            best_Q = Q
                            best_column_name = column.name
                    prev_value = value 
        else:
            for column in X.columns:
                sample = self.bins[column]
                column = X[column]
                for Q in sample:
                    ig = self.criterion(y, column, Q)
                    if ig > best_ig:
                        best_ig = ig
                        best_Q = Q
                        best_column_name = column.name
          
        return best_column_name, best_Q, best_ig    
    
    
    def build_leaf(self, X, y):
        return np.sum(y[X.index]) / len(y[X.index])
    
    def build_tree(self, X, y, current_depth):
        if self.leafs_cnt >= self.max_leafs:
            return self.build_leaf(X,y)
        
        if X.shape[0] <= 1 or len(np.unique(y)) <= 1:
            return self.build_leaf(X,y)
        
        split_column, Q, ig = self.get_best_split(X, y)
        node = TreeNode(split_column, Q, -1)
        
        
        if current_depth < self.max_depth and len(y) >= self.min_samples_split:

            left_indices = X[split_column] <= Q
            right_indices = X[split_column] > Q

            X_left = X[left_indices]
            y_left = y[left_indices]
            X_right = X[right_indices]
            y_right = y[right_indices]
    
            self.leafs_cnt += 1
            node.left = self.build_tree(X_left, y_left, current_depth + 1)
            node.right = self.build_tree(X_right, y_right, current_depth + 1)
            
            self.fi[split_column] += X.shape[0]/self.n*(
                self.metric(y) - 
                len(y_left)/len(y)*self.metric(y_left) -
                len(y_right)/len(y)*self.metric(y_right)
            )
            
            
        else:
            return self.build_leaf(X,y)
        return node
        
    def fit(self,X, y):
        self.n = len(y)
        self.leafs_cnt = 1
        self.fi = {key: 0 for key in X.columns}
        if self.bins != None and self.bins < X.shape[0] - 2:
            self.bins = self.get_bins(X)
        else:
            self.bins = None
        self.node = self.build_tree(X, y,0)
        
    def get_bins(self, X):
        bins = pd.DataFrame()
        for column in X.columns:
            sample = np.histogram(X[column], bins=self.bins)[1][1:-1]
            bins[column] = sample    
        return bins
        
    def predict(self, X):
        y = self.predict_proba(X)
        return y.apply(lambda x: 1 if x > 0.5 else 0)
            
    
    def predict_proba(self, X):
        self.y_predict = pd.Series(index=X.index)
        self.prediction(self.node, X)
        return self.y_predict
    
    def prediction(self, node, X):
        if type(node) == np.float64:
            self.y_predict[X.index] = node
            return
        
        left_indexes = X[node.split_column] <= node.split_value
        right_indexes = X[node.split_column] > node.split_value
        self.prediction(node.left, X[left_indexes])
        self.prediction(node.right, X[right_indexes]) 
            
    def print_tree(self, node):
        if type(node) == np.float64:
            print(node)
            return
        print(node.split_column, node.split_value)
        self.print_tree(node.left)
        self.print_tree(node.right)
 

In [90]:
class MyForestClf:
    
    def __init__(self, n_estimators=10, max_features=0.5, max_samples=0.5, random_state=42,
                 max_depth=5, min_samples_split=2, max_leafs=20, bins=16, criterion='entropy', oob_score=None):
        self.n_estimators = n_estimators    
        self.max_features = max_features
        self.max_samples = max_samples
        self.random_state = random_state
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs if max_leafs > 1 else 2
        self.bins = bins
        self.criterion = criterion
        self.leafs_cnt = 0
        self.forest = []
#         self.fi = None
        self.oob_score_ = 0
        if oob_score is not None:
            self.oob_score = globals()[oob_score]
        
    def __str__(self):
        return f"MyForestClf class: n_estimators={self.n_estimators}, max_features={self.max_features}," + f"max_samples={self.max_samples}, max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}, bins={self.bins}, criterion={self.criterion}, random_state={self.random_state}"
       
    def fit(self, X, y):
        random.seed(self.random_state)
        init_cols = X.columns.tolist()
        init_rows_cnt = X.shape[0]
#         self.fi = {key: 0 for key in X.columns}
        
        cols_smpl_cnt = int(np.round(len(X.columns) * self.max_features))
        rows_smpl_cnt = int(np.round(X.shape[0] * self.max_samples))
        
        for i in range(self.n_estimators):
            cols_idx = random.sample(init_cols, cols_smpl_cnt)
            rows_idx = random.sample(range(init_rows_cnt), rows_smpl_cnt)
            
            forest_tree = MyTreeClf(max_depth=self.max_depth, min_samples_split=self.min_samples_split,
                                   max_leafs=self.max_leafs, bins=self.bins, criterion=self.criterion)
            forest_tree.fit(X.loc[rows_idx, cols_idx], y[rows_idx])
            
            oob_rows_idx = X.index.difference(rows_idx)
            y_prediction = forest_tree.predict(X.loc[oob_rows_idx, cols_idx])
            if self.oob_score is not None:
                self.oob_score_ += self.oob_score(y[oob_rows_idx], y_prediction)
            
            self.leafs_cnt += forest_tree.leafs_cnt
#             for key in forest_tree.fi:
#                 self.fi[key] += forest_tree.fi[key]
            
            self.forest.append(forest_tree)
        if self.oob_score is not None:
            self.oob_score_ = self.oob_score_ / len(self.forest)
            
    def predict(self, X, type):
        y_prediction = np.zeros(X.shape[0])
        if type == 'mean':
            for forest_tree in self.forest:
                y_prediction += forest_tree.predict_proba(X)
            return (y_prediction/len(self.forest)).apply(lambda x: 1 if x > 0.5 else 0)  
        elif type == 'vote':
            for forest_tree in self.forest:
                y_prediction += forest_tree.predict(X)
            return (y_prediction/len(self.forest)).apply(lambda x: 1 if x >= 0.5 else 0)
        
    def predict_proba(self, X):
        y_prediction = np.zeros(X.shape[0])
        for forest_tree in self.forest:
            y_prediction += forest_tree.predict_proba(X)
        return y_prediction/len(self.forest)

In [91]:
forest = MyForestClf(n_estimators=6, max_depth=2, max_features=0.6, max_samples=0.5, criterion='entropy', oob_score='f1')

In [92]:
forest.fit(X,y)

In [93]:
forest.leafs_cnt

23

In [94]:
forest.predict(X, type='mean')

0       0
1       0
2       0
3       0
4       1
       ..
1367    1
1368    1
1369    1
1370    1
1371    1
Length: 1372, dtype: int64

In [96]:
forest.oob_score_

0.7253524836599691